# skm performance measurements
- `skm.fit_minibatch(X)` is comparable to `skm.fit(X)`
- `skm.fit_minibatch(X_batches[i])` is significantly slower than `skm.fit(X)`
- Stick with `skm.fit(X)`

In [ ]:
figsize('inline_short')

from collections import OrderedDict
import glob
import itertools
import json
import random
from typing import List

from skm import SKM

import metadata
from util import *

In [ ]:
# Load recordings paths
recs_paths = pd.DataFrame([
    OrderedDict(
        path=path,
    )
    for path in flatten(glob.glob(f'{data_dir}/{g}') for g in [
        'peterson-field-guide/*/audio/*',
        'recordings/*',
        'recordings-new/*',
    ])
    if not os.path.isdir(path)
])
display(
    recs_paths.shape,
    pd.concat([
        recs_paths[:5],
        recs_paths[-5:],
    ]),
    recs_paths.path.map(lambda path: os.path.relpath(path, data_dir).split('/')[0]).value_counts(),
)

(478, 1)

,path
0,/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220301 Song.mp3
1,/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220321 Song.mp3
2,/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220351 Song.mp3
3,/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220371 Song (2 types).mp3
4,"/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220391 Song, Chatter, Snarl, etc..mp3"
473,/Users/danb/hack/bubo/data/recordings-new/x 20180414 unknown- long rolling finchy- GG Park Botanical Gardens.wav
474,/Users/danb/hack/bubo/data/recordings-new/x mixed 20170514 various (community garden by christopher playground).wav
475,/Users/danb/hack/bubo/data/recordings-new/x mixed 20180415 mixed sparrows- glen canyon.wav
476,"/Users/danb/hack/bubo/data/recordings-new/x wren 20161023 wren sp, alert.wav"
477,/Users/danb/hack/bubo/data/recordings-new/x wren 20161023 wren sp.wav


peterson-field-guide    341
recordings-new           86
recordings               51
Name: path, dtype: int64

In [ ]:
# Load audio data for each recording
recs_audio = (recs_paths
    # [lambda df: df.species == 'WIWA'].reset_index(drop=True)  # For faster dev
    # [:10]  # For faster dev
    .assign(audio=lambda df: df.reset_index(drop=True).pipe(df_apply_with_progress, f=lambda rec:
        load_audio(rec.path, cache=True, verbose=False)
    ))
)
display(
    recs_audio.shape,
    recs_audio.audio[:5],
)

Progress[0/478, .../s, ETA ...]: {'path': '/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220301 Song.mp3'}


Progress[478/478, 346/s, took 1.38s]


(478, 2)

0                          peterson-field-guide/bewwre/audio/43220301 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 10.632s long
1                          peterson-field-guide/bewwre/audio/43220321 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 17.162s long
2                          peterson-field-guide/bewwre/audio/43220351 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 49.136s long
3                peterson-field-guide/bewwre/audio/43220371 Song (2 types): 1 channels, 16 bit, sampled @ 22.05 kHz, 29.753s long
4    peterson-field-guide/bewwre/audio/43220391 Song, Chatter, Snarl, etc.: 1 channels, 16 bit, sampled @ 22.05 kHz, 27.011s long
Name: audio, dtype: object

In [ ]:
recs = (recs_audio
    .pipe(df_apply_with_progress, f=lambda row: pd.Series(dict(
        **row.to_dict(),
        name=row.audio.name,
        samples=row.audio.to_numpy_array(),
        **lookup_audio_metadata(row.audio),
    )))
    .drop(columns=['path'])
    .pipe(df_reorder_cols, first=['name', 'source', 'species', 'basename'], last=['audio', 'samples'])
)
display(
    recs.shape,
    recs[:5],
    recs.fillna('').groupby(['source', 'species'])[['name']].count(),
)

Progress[0/478, .../s, ETA ...]: {'path': '/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220301 Song.mp3', 'audio': peterson-field-guide/bewwre/audio/43220301 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 10.632s long}


Progress[0/478, .../s, ETA ...]: {'path': '/Users/danb/hack/bubo/data/peterson-field-guide/bewwre/audio/43220301 Song.mp3', 'audio': peterson-field-guide/bewwre/audio/43220301 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 10.632s long}


Progress[478/478, 489/s, took 0.978s]


(478, 7)

,name,source,species,basename,species_query,audio,samples
0,peterson-field-guide/bewwre/audio/43220301 Song,peterson-field-guide,BEWR,43220301 Song,bewwre,"peterson-field-guide/bewwre/audio/43220301 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 10.632s long",[ 0 0 -1 ... 0 0 0]
1,peterson-field-guide/bewwre/audio/43220321 Song,peterson-field-guide,BEWR,43220321 Song,bewwre,"peterson-field-guide/bewwre/audio/43220321 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 17.162s long",[ 0 0 0 ... -1 0 1]
2,peterson-field-guide/bewwre/audio/43220351 Song,peterson-field-guide,BEWR,43220351 Song,bewwre,"peterson-field-guide/bewwre/audio/43220351 Song: 1 channels, 16 bit, sampled @ 22.05 kHz, 49.136s long",[0 0 0 ... 1 0 0]
3,peterson-field-guide/bewwre/audio/43220371 Song (2 types),peterson-field-guide,BEWR,43220371 Song (2 types),bewwre,"peterson-field-guide/bewwre/audio/43220371 Song (2 types): 1 channels, 16 bit, sampled @ 22.05 kHz, 29.753s long",[ 0 -1 1 ... 0 1 -1]
4,"peterson-field-guide/bewwre/audio/43220391 Song, Chatter, Snarl, etc.",peterson-field-guide,BEWR,"43220391 Song, Chatter, Snarl, etc.",bewwre,"peterson-field-guide/bewwre/audio/43220391 Song, Chatter, Snarl, etc.: 1 channels, 16 bit, sampled @ 22.05 kHz, 27.011s long",[1 1 0 ... 0 0 0]


name
source               species      
peterson-field-guide BCCH       19
                     BCTI       20
                     BEWR       19
                     BOCH       12
                     BUSH       12
                     CACH       18
                     CACW       14
                     CANW       15
                     CARW       19
                     CEDW        7
                     HOWR       20
                     MAWR       20
                     RBNU       16
                     ROWR       13
                     SEWR       11
                     SOSP       19
                     TUTI       26
                     WBNU       17
                     WCSP       20
                     WIWA       15
                     WIWR        9
recordings                      51
recordings-new                  34
                     BEWR        3
                     CATH        2
                     CEDW        1
                     GRCA        1
                     MAWR        1
                     OCWA        1
                     SOSP        8
                     STJA        1
                     WCSP       18
                     WIWA       16

In [ ]:
from sp14.model import *
model = Model(verbose_params=True)

_recs = [Recording(**row._asdict()) for row in recs.itertuples(index=False)]
random.seed(1); random.shuffle(_recs)

[00:50:17.999] init:params
  rec_sample_rate: 22050 Hz
  spectro_f_min: 1000 Hz
    f_max: 11025 Hz
  spectro_f_bins (f): 40 freq bins
  spectro_hop_length: 256 samples (11.6 ms)
  spectro_frame_length: 512 samples (23.2 ms)
    frame_overlap: 50% overlap (256 samples)
    frames/s (t/s): 86.1 samples/s
  spectro_frame_window: '''hann'''
  norm: '[TODO]'
  patch_length (p): 4 frames (46.4 ms)
  proj_skm_pca_var: 99% variance
  proj_skm_k: 500 clusters
  agg_funs: '[''mean'', ''std'', ''max'']'
    a: 3 aggs
    features: 1500 features
  class_knn_k: 3
[00:50:18.001] init:pipeline
  spectro: (f, t)   (40, 86.1/s)
  patch  : (f*p, t) (40*4, 86.1/s)
  proj   : (k, t)   (500, 86.1/s)
  agg    : (k, a)   (500, 3)
  feat   : (k*a,)   (1500,)


In [ ]:
%%time
patches = model.patches(_recs[:10])
skm = SKM(model.proj_skm_k)
skm_X = np.concatenate(patches, axis=1)
print(f'[skm_X.shape: {skm_X.shape}]')
skm.fit(skm_X)

[00:51:02.845] patches:recs
  len(recs): 10
  (samples,): [494208, 438336, 617472, 172800, 1268729, 1541030, 451008, 661824, 548352, 801792]


[00:51:03.347] patches:spectros
  (f, t): [[40, 1929], [40, 1711], [40, 2411], [40, 674], [40, 4954], [40, 6018], [40, 1760], [40, 2584], [40, 2141], [40, 3131]]
[00:51:03.420] patches:patches
  (f*p, t): [[160, 1926], [160, 1708], [160, 2408], [160, 671], [160, 4951], [160, 6015], [160, 1757], [160, 2581], [160, 2138], [160, 3128]]
[skm_X.shape: (160, 27283)]


CPU times: user 11.4 s, sys: 3.67 s, total: 15.1 s
Wall time: 4.07 s


In [ ]:
%%time
patches = model.patches(_recs[:10])
skm = SKM(model.proj_skm_k)
skm_X = np.concatenate(patches, axis=1)
print(f'[skm_X.shape: {skm_X.shape}]')
skm.fit_minibatch(skm_X)

[00:51:06.970] patches:recs
  len(recs): 10
  (samples,): [494208, 438336, 617472, 172800, 1268729, 1541030, 451008, 661824, 548352, 801792]


[00:51:07.484] patches:spectros
  (f, t): [[40, 1929], [40, 1711], [40, 2411], [40, 674], [40, 4954], [40, 6018], [40, 1760], [40, 2584], [40, 2141], [40, 3131]]
[00:51:07.562] patches:patches
  (f*p, t): [[160, 1926], [160, 1708], [160, 2408], [160, 671], [160, 4951], [160, 6015], [160, 1757], [160, 2581], [160, 2138], [160, 3128]]
[skm_X.shape: (160, 27283)]


EPOCH: 1 CHANGE: 1.0000
EPOCH: 2 CHANGE: 0.5347


EPOCH: 3 CHANGE: 0.3321


EPOCH: 4 CHANGE: 0.2038
EPOCH: 5 CHANGE: 0.1310


...

EPOCH: 12 CHANGE: 0.0254


EPOCH: 13 CHANGE: 0.0207
EPOCH: 14 CHANGE: 0.0191


EPOCH: 15 CHANGE: 0.0174
EPOCH: 16 CHANGE: 0.0141


EPOCH: 17 CHANGE: 0.0125
EPOCH: 18 CHANGE: 0.0107


EPOCH: 19 CHANGE: 0.0094
CPU times: user 12.6 s, sys: 4.26 s, total: 16.8 s
Wall time: 4.7 s


In [ ]:
%%time
patches = model.patches(_recs[:10])
skm = SKM(model.proj_skm_k)
skm_X = np.concatenate(patches, axis=1)
for i in range(1 + skm_X.shape[1] // 1000):
    batch = skm_X[:, 1000*i : 1000*(i+1)]
    print(f'[batch.shape: {batch.shape}]')
    skm.fit_minibatch(skm_X)

[00:50:39.522] patches:recs
  len(recs): 10
  (samples,): [494208, 438336, 617472, 172800, 1268729, 1541030, 451008, 661824, 548352, 801792]


[00:50:40.056] patches:spectros
  (f, t): [[40, 1929], [40, 1711], [40, 2411], [40, 674], [40, 4954], [40, 6018], [40, 1760], [40, 2584], [40, 2141], [40, 3131]]
[00:50:40.175] patches:patches
  (f*p, t): [[160, 1926], [160, 1708], [160, 2408], [160, 671], [160, 4951], [160, 6015], [160, 1757], [160, 2581], [160, 2138], [160, 3128]]
[batch.shape: (160, 1000)]


EPOCH: 1 CHANGE: 1.0000
EPOCH: 2 CHANGE: 0.5245


EPOCH: 3 CHANGE: 0.3202
EPOCH: 4 CHANGE: 0.1866


EPOCH: 5 CHANGE: 0.1215
EPOCH: 6 CHANGE: 0.0894


...

EPOCH: 1 CHANGE: 1.0000
EPOCH: 2 CHANGE: 0.0005
[batch.shape: (160, 1000)]


EPOCH: 1 CHANGE: 1.0000


EPOCH: 2 CHANGE: 0.0003
[batch.shape: (160, 1000)]


EPOCH: 1 CHANGE: 1.0000
EPOCH: 2 CHANGE: 0.0003
[batch.shape: (160, 283)]


EPOCH: 1 CHANGE: 1.0000
EPOCH: 2 CHANGE: 0.0002
CPU times: user 53.8 s, sys: 17.9 s, total: 1min 11s
Wall time: 19.9 s
